In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sjoert.stellar
import pandas as pd
import os
from tqdm import tqdm
from astropy import coordinates as coord
from scipy.optimize import curve_fit
import forced_phot as fp #import flux_jy, chi2, gaussian, chi2_peak_finder
from scipy.constants import h,c,k
import json

sjoertpath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares'

The below shouldn't be in the class but should be the loop in which the class is called.

In [33]:
test_ztf = 'ZTF18abjjkeo'
cwd = os.getcwd()
dir = cwd
while not dir.endswith('First Research Project\\Data'):
    cwd = os.path.split(cwd)[0]
    dir = cwd + "\\Data"
banlist = set(['DOWNLOADED','EMPTY','dump'])
iterlist = sorted(set(os.listdir(dir)) - banlist)

for folder in iterlist:
    folder_dir = os.path.join(dir,folder)
    for ztf_name in os.listdir(folder_dir):
        # call the class
        pass



False

In [3]:
class ZTF_forced_phot:
    def __init__(self,ztf_name): #ztf_name may be simply the name or the path leading to where the relevant data is stored if available.
        if "\\" in ztf_name: #ergo a directory was given instead of a name
            ztf_dir = ztf_name #make self(?)
            ztf_name = os.path.split(ztf_dir)[-1]
            pass # don't do the part of finding where the data is stored
        else:
            cwd = os.getcwd()
            dir = cwd
            while not dir.endswith('First Research Project\\Data'):
                cwd = os.path.split(cwd)[0]
                dir = cwd + "\\Data"
            banlist = set(['DOWNLOADED','EMPTY','dump'])
            iterlist = sorted(set(os.listdir(dir)) - banlist)

            for folder in iterlist:
                folder_dir = os.path.join(dir,folder)
                if ztf_name in os.listdir(folder_dir):
                    ztf_dir = os.path.join(folder_dir,ztf_name) #make self (?)
                    break
        
        clean_data = pd.read_json(os.path.join(ztf_dir,ztf_name,"_clean_data.json")) #make self
        clean_data.sort_values('time',inplace=True) #sort the data by time for plotting reasons later on
        no_i_mask = clean_data['filter'] != 'ZTF_i' #create the ZTf_i mask
        flux,err = fp.flux_jy(clean_data) #convert the flux and its errors to uJY
        clean_data['flux'] = flux.values
        clean_data['flux_unc'] = err.values

        time_zeropoint = np.min(clean_data['time'][no_i_mask]) #find the zeropoint for time to be used here
        time_mjd = clean_data['time'] - time_zeropoint
        clean_data['time'] = time_mjd.values #transform time column

        with open(os.path.join(ztf_dir,ztf_name,"_clean_log.json")) as f:
            logfile = json.read(f)

        columns = ['sindex', 'field', 'ccdid', 'qid', 'filter', 'pid', 'infobitssci', 'sciinpseeing', 'scibckgnd', 'scisigpix', 'zpmaginpsci', 'zpmaginpsciunc', 'zpmaginpscirms', 'clrcoeff', 'clrcoeffunc', 'ncalmatches', 'exptime', 'adpctdif1', 'adpctdif2', 'diffmaglim', 'zpdiff', 'programid', 'jd', 'rfid', 'forcediffimflux', 'forcediffimfluxunc', 'forcediffimsnr', 'forcediffimchisq', 'forcediffimfluxap', 'forcediffimfluxuncap', 'forcediffimsnrap', 'aperturecorr', 'dnearestrefsrc', 'nearestrefmag', 'nearestrefmagunc', 'nearestrefchi', 'nearestrefsharp', 'refjdstart', 'refjdend', 'procstatus']
        dtypes = [(columns[x],float) for x in range(len(columns))]
        dtypes[4] = ('filter',r'U8')
        for file in os.listdir(ztf_dir):
            if 'batchfp' in file:
                batchrq_string = file
        raw_data = pd.DataFrame(np.genfromtxt(os.path.join(ztf_dir,batchrq_string),skip_header=53,dtype=dtypes))
  
        err = np.clip(err,0.01*flux,np.inf) # clip the errors to be at least 1% of the flux


        self.ztf_name = ztf_name
        self.no_i_mask = no_i_mask
        self.time_zeropoint = time_zeropoint
        self.clean_data = clean_data
        self.logfile = logfile
        self.raw_data = raw_data


